# Regressão Linear Multivariada - Trabalho

## Estudo de caso: Qualidade de Vinhos

Nesta trabalho, treinaremos um modelo de regressão linear usando descendência de gradiente estocástico no conjunto de dados da Qualidade do Vinho. O exemplo pressupõe que uma cópia CSV do conjunto de dados está no diretório de trabalho atual com o nome do arquivo *winequality-white.csv*.

O conjunto de dados de qualidade do vinho envolve a previsão da qualidade dos vinhos brancos em uma escala, com medidas químicas de cada vinho. É um problema de classificação multiclasse, mas também pode ser enquadrado como um problema de regressão. O número de observações para cada classe não é equilibrado. Existem 4.898 observações com 11 variáveis de entrada e 1 variável de saída. Os nomes das variáveis são os seguintes:

1. Fixed acidity.
2. Volatile acidity.
3. Citric acid.
4. Residual sugar.
5. Chlorides.
6. Free sulfur dioxide. 
7. Total sulfur dioxide. 
8. Density.
9. pH.
10. Sulphates.
11. Alcohol.
12. Quality (score between 0 and 10).

O desempenho de referencia de predição do valor médio é um RMSE de aproximadamente 0.148 pontos de qualidade.

Utilize o exemplo apresentado no tutorial e altere-o de forma a carregar os dados e analisar a acurácia de sua solução. 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from csv import reader
from random import randrange
from math import sqrt
from sklearn.metrics import accuracy_score, classification_report

In [2]:
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return yhat

def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    # print ('Coeficiente Inicial={0}' % (coef))
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            sum_error += error**2
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] - l_rate * error * row[i] 
        #print(('epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error)))
    return coef

In [3]:
df = pd.read_csv('winequality-white.csv',names = ['Fixed acidity','Volatile acidity','Citric acid','Residual sugar','Chlorides','Free sulfur dioxide','Total sulfur dioxide','Density','pH','Sulphates','Alcohol','Quality'])
df.head()

,Fixed acidity,Volatile acidity,Citric acid,Residual sugar,Chlorides,Free sulfur dioxide,Total sulfur dioxide,Density,pH,Sulphates,Alcohol,Quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
for f in ['Fixed acidity','Volatile acidity','Citric acid','Residual sugar','Chlorides','Free sulfur dioxide','Total sulfur dioxide','Density','pH','Sulphates','Alcohol']:
    df[f] = df[f].astype('category')
    df[f] = df[f].cat.codes

In [5]:
features = ['Fixed acidity','Volatile acidity','Citric acid','Residual sugar','Chlorides','Free sulfur dioxide','Total sulfur dioxide','Density','pH','Sulphates','Alcohol']
X = df.get(features)
X

,Fixed acidity,Volatile acidity,Citric acid,Residual sugar,Chlorides,Free sulfur dioxide,Total sulfur dioxide,Density,pH,Sulphates,Alcohol
0,31,36,36,302,34,57,146,878,23,22,5
1,23,42,34,17,38,13,106,471,53,26,12
2,43,38,40,101,39,33,69,560,49,21,24
3,34,28,32,128,47,59,163,601,42,17,21
4,34,28,32,128,47,59,163,601,42,17,21
5,43,38,40,101,39,33,69,560,49,21,24
6,22,46,16,103,34,33,110,544,41,24,15
7,31,36,36,302,34,57,146,878,23,22,5
8,23,42,34,17,38,13,106,471,53,26,12
9,43,26,43,15,33,30,102,453,45,22,43


In [6]:
Y = df['Quality'].values

Y

array([6, 6, 6, ..., 6, 7, 6])

In [7]:
X = MinMaxScaler().fit_transform(X)
ds = np.column_stack([X, Y])
print(ds)

[[ 0.46268657  0.29032258  0.41860465 ...,  0.28205128  0.04901961  6.        ]
 [ 0.34328358  0.33870968  0.39534884 ...,  0.33333333  0.11764706  6.        ]
 [ 0.64179104  0.30645161  0.46511628 ...,  0.26923077  0.23529412  6.        ]
 ..., 
 [ 0.3880597   0.24193548  0.22093023 ...,  0.29487179  0.10784314  6.        ]
 [ 0.20895522  0.32258065  0.34883721 ...,  0.19230769  0.82352941  7.        ]
 [ 0.28358209  0.19354839  0.44186047 ...,  0.11538462  0.6372549   6.        ]]


In [8]:
train, test = train_test_split(ds, test_size = 0.3)

In [9]:
l_rate = 0.0001
n_epoch = 100

coefs = coefficients_sgd(train, l_rate, n_epoch)
print(coefs)

[3.4029300305905181, 0.80443416035269022, -0.1295338519631013, 0.69928900238763125, 0.26449097492915419, 0.2411940584024376, 0.56376844161857964, 0.20005152588438657, 0.1436303923927901, 1.1744885211676466, 0.64679462954705169, 2.0577630282783157]


In [10]:
predictions = list()
for row in test:
    predicted = predict(row, coefs)
    predictions.append(predicted)

In [11]:
y_test = list()
for i in range(1470):
    y_test.append(test[i][11])

In [12]:
predicted = predictions

In [15]:
soma = 0
for i in range(len(y_test)):
    erro = predicted[i] - y_test[i]
    soma += (erro**2)
    erroMedio = soma/float(len(y_test))
acc = sqrt(erroMedio)   

In [16]:
acc

0.8120004794092449